In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *

pd.options.mode.chained_assignment = None

shots_DIR = "../data/ShotLocationData/"

In [ ]:
league = "NBA"
season = "2023"
df = pd.read_parquet(shots_DIR + league + "_Shot_Details_" + season + ".parquet")
teams_dict, teams_list = get_teams(league="NBA")
all_shots = df.copy()

In [ ]:
all_shots.columns = map(str.lower, all_shots.columns)
all_shots = all_shots.rename(columns={"shot_made_flag":"shot_made"})
all_shots["loc_x"] = all_shots["loc_x"]/10
all_shots["loc_y"] = all_shots["loc_y"]/10
all_shots['fg2'] = np.where(all_shots['shot_type'] == '2PT Field Goal', 1, 0)
all_shots['fg3'] = np.where(all_shots['shot_type'] == '3PT Field Goal', 1, 0)
all_shots["points"] = (2*all_shots['fg2'] + 3*all_shots['fg3'])*all_shots["shot_made"]
all_shots["game_date"] = pd.to_datetime(all_shots["game_date"], format="%Y%m%d")
# shots = all_shots.loc[all_shots['shot_distance'] <= 32].reset_index(drop=True)

In [ ]:
shots1 = all_shots[ all_shots["shot_zone_basic"].isin(["Restricted Area",'Above the Break 3','Left Corner 3', 'Right Corner 3'])].reset_index(drop=True)
date_d = pd.to_datetime(dt.date(2023,12,19))
shots = shots1.query(f"(game_date < '{date_d}') & (team_name == 'Memphis Grizzlies')")
# shots = shots1[shots1["game_date"] >= date_d]

In [ ]:
fg0 = (shots
      .groupby(['team_id'])[['points', 'shot_made']]
      .agg({'points': ['sum', 'count'],
           'shot_made': ['sum'],}))
fg0.columns = ['PTS', 'number_of_shots', 'number_of_makes']
fg = fg0.reset_index()
fg['eFG'] = np.round(fg['PTS']/fg['number_of_shots']/2, 3)
fg = fg.rename(columns = {'number_of_shots': 'FGA',
                           'number_of_makes': 'FGM'})
fg["team_id"] = fg["team_id"].map(teams_dict)
fg = fg.rename(columns={"team_id":"Team"})
# fg = fg[['player_id', 'FGA', 'FGM', 'eFG', 'xeFG', 'PTS', 'xPTS', 'Shot_Making']]
fg = fg.sort_values("FGA",ascending=False)

In [ ]:
fg

In [ ]:
dfgdfg

In [ ]:
%reload_ext rpy2.ipython

In [ ]:
%%R -i fg
library(tidyverse)
library(gt)
df <- fg
df %>% 
  gt()%>%
  tab_header(
    title = md("**Most MoreyBall Shot Attempts 2023-24**"),
    subtitle = "Moreyball = Shots at Rim + 3PT Shots" 
    ) %>%
    data_color(columns = c(eFG,FGM,FGA), palette = "PRGn") %>%
    # cols_align(align = "center",columns = c(Shot_Making))  %>%
    cols_label(
      eFG = "eFG%"
    ) %>%
    fmt_percent(
      columns = eFG,
      decimals = 1
    ) %>%
    tab_options(
        table.background.color = "floralwhite",
        column_labels.font.size = 12,
        column_labels.font.weight = 'bold',
        row_group.font.weight = 'bold',
        row_group.background.color = "#E5E1D8",
        table.font.size = 10,
        heading.title.font.size = 16,
        heading.subtitle.font.size = 12.5,
        table.font.names = "Consolas", 
        data_row.padding = px(2)
    ) %>% 
    tab_source_note(
    source_note = "eFG%: Effective Field Goal %")  %>% 
    tab_source_note(
    source_note = "@SravanNBA | Source: nba.com/stats" ) %>% gtsave("../figs/team_leaders/moreyball.png",zoom=5) 